## Notebook for machine learning methods for rf-based detection & classification
-  exploring, SVM, Logistic regression with PSD

In [1]:
import os
import numpy as np
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

from helper_functions import *
from latency_helpers import *
from loading_functions import *

# DroneDetect Dataset

### Load Features

In [65]:
feat_folder = '../Features/'
feat_name = 'PSD'
seg_len = 50
n_per_seg = 256
interferences = ['WIFI', 'BLUE', 'BOTH', 'CLEAN']
output_name = 'drones'
feat_format = 'ARR'

dataset = DroneDetectTorch(feat_folder, feat_name, seg_len, n_per_seg, feat_format,
                                output_name, interferences)

# Xs_arr, y_arr = load_dronedetect_features(feat_folder, feat_name, seg_len, 
#                                           n_per_seg, output_feat,interferences)

Directory Name:  ../Features/ARR_PSD_256_50/


In [66]:
print('dataset size', len(dataset))
print('shape of each item', dataset.__getitem__(0)[0].shape)

dataset size 15591
shape of each item (256,)


In [52]:
# load all to local memory
i_all = list(range(len(dataset)))
X_use, y_use = dataset.__getitem__(i_all)

## Train Test split

In [53]:
# split data into K-fold
k_fold = 5
kf = KFold(n_splits=k_fold, random_state=1, shuffle=True)

# model parameters
Cs=list(map(lambda x:pow(2,x),range(-3,10,2)))
gammas=list(map(lambda x:pow(2,x),range(-3,10,2)))

## SVM

In [54]:
### Not running for now- takes long time for gridsearchcv even with n_jobs -1
### Run all combination of data parameters with fixed parameters first

# best_params_ls = []
# acc_ls = []
# f1_ls = []
# runt_ls = []
# parameters = {'C':Cs, 'gamma':gammas}
# for train_ix, test_ix in kf.split(X_use):
    
#     # find the optimal hypber parameters
#     svc = svm.SVC(kernel='rbf')
#     clf = GridSearchCV(svc, parameters, n_jobs=1)
#     clf.fit(X_use[train_ix], y_use[train_ix])
    
#     print(clf.best_params_)
#     best_params_ls.append(clf.best_params_)
    
#     # predict on the test data
# #     y_pred = clf.predict(Xs_use[test_ix])
#     y_pred, runtimes = atomic_benchmark_estimator(clf, X_use[test_ix], '<U3', verbose=False) # predict & measure time
#     runt_ls.append(np.mean(runtimes))
    
#     acc = accuracy_score(y_use[test_ix], y_pred)
#     f1 = f1_score(y_use[test_ix], y_pred, average='weighted')
#     print('Accuracy: {:.3},\t F1: {:.3}'.format(acc,f1))
#     acc_ls.append(acc)
#     f1_ls.append(f1)

# # print(feat_name+': SVM K-fold average test acc:', np.mean(acc_ls), 'F1:', np.mean(f1_ls), 'Run-time:', np.mean(runt_ls)*1e3,'ms')
# out_msg = feat_name+': SVM K-fold average test acc: {:.3}, F1: {:.3}, Run-time: {:.3}ms'.format(np.mean(acc_ls), np.mean(f1_ls), np.mean(runt_ls)*1e3)
# print(out_msg)

In [55]:
# print out the size of the support vectors
# clf.best_estimator_.support_vectors_

In [56]:
Cs

[0.125, 0.5, 2, 8, 32, 128, 512]

## SVM with fixed hyperparameters

In [57]:
C_fix = 1 ##default params
gamma_fix = 'scale'

acc_ls = []
f1_ls = []
runt_ls = []

k_fold = 5
cv = KFold(n_splits=k_fold, random_state=None, shuffle=True)

for train_ix, test_ix in cv.split(X_use):
    svc = svm.SVC(kernel='rbf', C=C_fix, gamma = gamma_fix)
    svc.fit(X_use[train_ix], y_use[train_ix])
    # predict on the test data
    y_pred, runtimes = atomic_benchmark_estimator(svc, X_use[test_ix], output_type='<U3', verbose=False)
    runt_ls.append(np.mean(runtimes))
    
    acc = accuracy_score(y_use[test_ix], y_pred)
    f1 = f1_score(y_use[test_ix], y_pred, average='weighted')
    print('Accuracy: {:.3},\t F1: {:.3}'.format(acc,f1))
    acc_ls.append(acc)
    f1_ls.append(f1)

out_msg = feat_name+': SVM K-fold average test acc: {:.2}, F1: {:.2}, Run-time: {:.2}ms'.format(np.mean(acc_ls), np.mean(f1_ls), np.mean(runt_ls)*1e3)
print(out_msg)

Accuracy: 0.886,	 F1: 0.884
Accuracy: 0.888,	 F1: 0.886
Accuracy: 0.89,	 F1: 0.888
Accuracy: 0.877,	 F1: 0.876
Accuracy: 0.883,	 F1: 0.881
PSD: SVM K-fold average test acc: 0.88, F1: 0.88, Run-time: 1.2ms


In [59]:
print(svc.support_vectors_.shape)

(4803, 256)


## Save Model

In [67]:
# feat_name = 'PSD'
# seg_len = 10
# n_per_seg = 512
import pickle
to_train_all = True # whether to retrain using all the data
model_name = 'SVM'+'_'+str(feat_name)+'_'+str(n_per_seg)+'_'+str(seg_len)+'_'+str(int(to_train_all))
model_path = 'saved_models/'
print('Model saved as:', model_name)

Model saved as: SVM_PSD_256_50_1


In [68]:
if to_train_all:
    svc.fit(X_use, y_use)

pickle.dump(svc, open(model_path+model_name, 'wb'))


## Logistic Regression

In [ ]:
# best_params_ls = []
# score_ls = []

# parameters = {'C':[0.01,0.1,1,10,100,1000,10000]}

# for train_ix, test_ix in cv.split(Xs_arr):
    
#     # find the optimal hypber parameters
#     lr = LogisticRegression(max_iter=1000000)
#     clf = GridSearchCV(lr, parameters, n_jobs=1)
#     clf.fit(Xs_arr[train_ix], y_arr[train_ix])
    
#     print(clf.best_params_)
#     best_params_ls.append(clf.best_params_)
    
#     # predict on the test data
#     y_pred = clf.predict(Xs_arr[test_ix])
#     acc = accuracy_score(y_arr[test_ix], y_pred)
#     print(acc)
#     score_ls.append(acc)
    
# print(feat_file_name+': LR K-fold average test score:', np.mean(score_ls))

# Drone RF Dataset

In [7]:
# Load features
feat_folder = '../Features_DroneRF/'
feat_name = 'PSD'
highlow = 'H'
seg_len = 50
n_per_seg = 1024
Xs_arr, y_arr = load_dronerf_features(feat_folder, feat_name, seg_len, n_per_seg, highlow, 'bi')


## Apply normalization
X_norm = Xs_arr
for n in range(len(Xs_arr)):
    X_norm[n] = Xs_arr[n]/max(Xs_arr[n])
X_norm.shape
y_arr = y_arr.reshape(len(y_arr),)

Xs_use = X_norm # Use normalized features
Xs_use.shape

print('X shape:', Xs_arr.shape, ' y shape:', y_arr.shape)

100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 569.26it/s]

X shape: (1130, 513)  y shape: (1130,)


In [4]:
y_arr.max()

1

In [38]:
best_params_ls = []
acc_ls = []
f1_ls = []
runt_ls = []
parameters = {'C':Cs, 'gamma':gammas}
for train_ix, test_ix in cv.split(Xs_use):
    
    # find the optimal hypber parameters
    svc = svm.SVC(kernel='rbf')
    clf = GridSearchCV(svc, parameters, n_jobs=1)
    clf.fit(Xs_use[train_ix], y_arr[train_ix])
    
    print(clf.best_params_)
    best_params_ls.append(clf.best_params_)
    
    # predict on the test data
#     y_pred = clf.predict(Xs_use[test_ix])
    y_pred, runtimes = atomic_benchmark_estimator(clf, Xs_use[test_ix], 'int', verbose=False) # predict & measure time
    runt_ls.append(np.mean(runtimes))
    
    acc = accuracy_score(y_arr[test_ix], y_pred)
    f1 = f1_score(y_arr[test_ix], y_pred, average='weighted')
    print('Accuracy: {:.3},\t F1: {:.3}'.format(acc,f1))
    acc_ls.append(acc)
    f1_ls.append(f1)

# print(feat_name+': SVM K-fold average test acc:', np.mean(acc_ls), 'F1:', np.mean(f1_ls), 'Run-time:', np.mean(runt_ls)*1e3,'ms')
out_msg = feat_name+': SVM K-fold average test acc: {:.2}, F1: {:.2}, Run-time: {:.2}ms'.format(np.mean(acc_ls), np.mean(f1_ls), np.mean(runt_ls)*1e3)
print(out_msg)

ValueError: 
All the 245 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
245 fits failed with the following error:
Traceback (most recent call last):
  File "/home/kzhou/.pyenv/versions/3.8.2/envs/main/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/kzhou/.pyenv/versions/3.8.2/envs/main/lib/python3.8/site-packages/sklearn/svm/_base.py", line 182, in fit
    y = self._validate_targets(y)
  File "/home/kzhou/.pyenv/versions/3.8.2/envs/main/lib/python3.8/site-packages/sklearn/svm/_base.py", line 739, in _validate_targets
    raise ValueError(
ValueError: The number of classes has to be greater than one; got 1 class


## Try Model on Gamut data

In [2]:
data_path = '/home/kzhou/Data/S3/leesburg_worker1/Features/'
Xgamut = load_gamut_features(data_path, 'psd')

100%|███████████████████████████████████████| 115/115 [00:00<00:00, 2255.41it/s]


In [7]:
# normalize data
## Apply normalization
X_gamut_norm = Xgamut
for n in range(len(Xgamut)):
    X_gamut_norm[n] = Xgamut[n]/max(Xgamut[n])

X_gamut_norm.max()

1.0

In [8]:
# Feed data through trained SVM model
y_gamut_pred = clf.predict(X_gamut_norm)


In [9]:
y_gamut_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,

## Save trained model for testing

In [ ]:
bestmodel = clf.best_estimator_

In [ ]:
filename = 'test_save_model.sav'
pickle.dump(bestmodel, open(filename, 'wb'))